# Distinguishing countermeasures by output

In [1]:
import io
import random

from collections import Counter
from tqdm.auto import tqdm, trange

from pyecsca.ec.mod import mod
from pyecsca.ec.point import Point
from pyecsca.ec.model import ShortWeierstrassModel
from pyecsca.ec.params import load_params_ectester
from pyecsca.ec.mult import LTRMultiplier
from pyecsca.ec.countermeasures import GroupScalarRandomization, AdditiveSplitting, MultiplicativeSplitting, EuclideanSplitting, BrumleyTuveri

In [2]:
model = ShortWeierstrassModel()
coords = model.coordinates["projective"]

In [3]:
add = coords.formulas["add-2015-rcb"]
dbl = coords.formulas["dbl-2015-rcb"]
mult = LTRMultiplier(add, dbl, complete=False)

In [4]:
gsr = GroupScalarRandomization(mult)
asplit = AdditiveSplitting(mult)
msplit = MultiplicativeSplitting(mult)
esplit = EuclideanSplitting(mult)
bt = BrumleyTuveri(mult)

## 3n test

In [5]:
key3n = 0x20959f2b437de1e522baf6d814911938157390d3ea5118660b852ab0d5387006
params3n = load_params_ectester(io.BytesIO(b"0xc381bb0394f34b5ed061c9107b66974f4d0a8ec89b9fe73b98b6d1368c7d974d,0x5ca6c5ee0a10097af291a8f125303fb1a3e35e8100411902245d691e0e5cb497,0x385a5a8bb8af94721f6fd10b562606d9b9df931f7fd966e96859bb9bd7c05836,0x4616af1898b92cac0f902a9daee24bbae63571cead270467c6a7886ced421f5e,0x34e896bdb1337e0ae5960fa3389fb59c2c8d6c7dbfd9aac33a844f8f98e433ef,0x412b3e5686fbc3ca4575edb0292232702ae721a7d4a230cc170a5561aa70e00f,0x01"), "projective")
bits3n = params3n.full_order.bit_length()
point3n = Point(X=mod(0x4a48addb2e471767b7cd0f6f1d4c27fe46f4a828fc20f950bd1f72c939b36a84, params3n.curve.prime),
                Y=mod(0x13384d38c353f862832c0f067e46a3e510bb6803c20745dfb31929f4a18d890d, params3n.curve.prime),
                Z=mod(1, params3n.curve.prime), model=coords)

In [6]:
print(f"prime:\t0x{params3n.curve.prime:x}")
print(f"a:\t0x{params3n.curve.parameters['a']:x}")
print(f"b:\t0x{params3n.curve.parameters['b']:x}")
print(f"G:\t[0x{params3n.generator.X:x},\n\t 0x{params3n.generator.Y:x}]")
print(f"n:\t0x{params3n.order:x}")
print(f"3n:\t0x{3 * params3n.order:x}")
print(f"\nP:\t[0x{point3n.X:x},\n\t 0x{point3n.Y:x}]")

prime:	0xc381bb0394f34b5ed061c9107b66974f4d0a8ec89b9fe73b98b6d1368c7d974d
a:	0x5ca6c5ee0a10097af291a8f125303fb1a3e35e8100411902245d691e0e5cb497
b:	0x385a5a8bb8af94721f6fd10b562606d9b9df931f7fd966e96859bb9bd7c05836
G:	[0x4616af1898b92cac0f902a9daee24bbae63571cead270467c6a7886ced421f5e,
	 0x34e896bdb1337e0ae5960fa3389fb59c2c8d6c7dbfd9aac33a844f8f98e433ef]
n:	0x412b3e5686fbc3ca4575edb0292232702ae721a7d4a230cc170a5561aa70e00f
3n:	0xc381bb0394f34b5ed061c9107b66975080b564f77de69264451f0024ff52a02d

P:	[0x4a48addb2e471767b7cd0f6f1d4c27fe46f4a828fc20f950bd1f72c939b36a84,
	 0x13384d38c353f862832c0f067e46a3e510bb6803c20745dfb31929f4a18d890d]


In [7]:
def generate_scalars_mod3(rem, samples):
    scalars = []
    while True:
        scalar = random.randint(0, params3n.full_order)
        if scalar % 3 == rem:
            scalars.append(scalar)
        if len(scalars) == samples:
            break
    return scalars

def test_3n(countermeasure, scalars):
    ctr = Counter()
    for k in tqdm(scalars, leave=False):
        mult.init(params3n, point3n)
        kP = mult.multiply(k).to_affine()
        mult.init(params3n, point3n)
        knP = mult.multiply(k + params3n.full_order).to_affine()
        mult.init(params3n, point3n)
        k2nP = mult.multiply(k + 2 * params3n.full_order).to_affine()

        countermeasure.init(params3n, point3n)
        res = countermeasure.multiply(k)
        aff = res.to_affine()
        if aff.equals(kP):
            ctr["k"] += 1
        elif aff.equals(knP):
            ctr["k + 1n"] += 1
        elif aff.equals(k2nP):
            ctr["k + 2n"] += 1
        else:
            ctr[aff] += 1
    for name, count in sorted(ctr.items()):
        print(f"{name}:\t{count}")

def test_3n_fixed_scalar(countermeasure, samples):
    test_3n(countermeasure, [key3n for _ in range(samples)])

def test_3n_random_scalar(countermeasure, samples):
    test_3n(countermeasure, [random.randint(0, params3n.full_order) for _ in range(samples)])

def test_3n_random_scalar_projected(countermeasure, samples):
    print("k = 0 mod 3")
    test_3n(countermeasure, generate_scalars_mod3(0, samples))
    print()
    print("k = 1 mod 3")
    test_3n(countermeasure, generate_scalars_mod3(1, samples))
    print()
    print("k = 2 mod 3")
    test_3n(countermeasure, generate_scalars_mod3(2, samples))

### Fixed scalar experiments

#### Group scalar randomization

In [8]:
test_3n_fixed_scalar(gsr, 1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

k:	343
k + 1n:	339
k + 2n:	318


#### Additive splitting

In [9]:
test_3n_fixed_scalar(asplit, 1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

k:	520
k + 1n:	480


#### Multiplicative splitting

In [10]:
test_3n_fixed_scalar(msplit, 1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

k:	219
k + 1n:	554
k + 2n:	227


#### Euclidean splitting

In [11]:
test_3n_fixed_scalar(esplit, 1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

k:	1000


#### Brumley and Tuveri bit-length fixing

In [12]:
test_3n_fixed_scalar(bt, 1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

k + 2n:	1000


### Random scalar experiments

#### Group scalar randomization

In [13]:
test_3n_random_scalar(gsr, 1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

k:	315
k + 1n:	330
k + 2n:	355


#### Additive splitting

In [14]:
test_3n_random_scalar(asplit, 1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

k:	505
k + 1n:	495


#### Multiplicative splitting

In [15]:
test_3n_random_scalar(msplit, 1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

k:	345
k + 1n:	327
k + 2n:	328


#### Euclidean splitting

In [16]:
test_3n_random_scalar(esplit, 1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

k:	1000


#### Brumley and Tuveri bit-length fixing

In [17]:
test_3n_random_scalar(bt, 1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

k + 1n:	35
k + 2n:	965


### Random scalar experiments projected to scalar divisor classes mod 3

#### Group scalar randomization

In [18]:
test_3n_random_scalar_projected(gsr, 1000)

k = 0 mod 3


  0%|          | 0/1000 [00:00<?, ?it/s]

k:	349
k + 1n:	300
k + 2n:	351

k = 1 mod 3


  0%|          | 0/1000 [00:00<?, ?it/s]

k:	358
k + 1n:	329
k + 2n:	313

k = 2 mod 3


  0%|          | 0/1000 [00:00<?, ?it/s]

k:	310
k + 1n:	324
k + 2n:	366


#### Additive splitting

In [19]:
test_3n_random_scalar_projected(asplit, 1000)

k = 0 mod 3


  0%|          | 0/1000 [00:00<?, ?it/s]

k:	513
k + 1n:	487

k = 1 mod 3


  0%|          | 0/1000 [00:00<?, ?it/s]

k:	484
k + 1n:	516

k = 2 mod 3


  0%|          | 0/1000 [00:00<?, ?it/s]

k:	518
k + 1n:	482


#### Multiplicative splitting

In [20]:
test_3n_random_scalar_projected(msplit, 1000)

k = 0 mod 3


  0%|          | 0/1000 [00:00<?, ?it/s]

k:	564
k + 1n:	213
k + 2n:	223

k = 1 mod 3


  0%|          | 0/1000 [00:00<?, ?it/s]

k:	216
k + 1n:	209
k + 2n:	575

k = 2 mod 3


  0%|          | 0/1000 [00:00<?, ?it/s]

k:	231
k + 1n:	564
k + 2n:	205


#### Euclidean splitting

In [21]:
test_3n_random_scalar_projected(esplit, 1000)

k = 0 mod 3


  0%|          | 0/1000 [00:00<?, ?it/s]

k:	1000

k = 1 mod 3


  0%|          | 0/1000 [00:00<?, ?it/s]

k:	1000

k = 2 mod 3


  0%|          | 0/1000 [00:00<?, ?it/s]

k:	1000


#### Brumley and Tuveri bit-length fixing

In [22]:
test_3n_random_scalar_projected(bt, 1000)

k = 0 mod 3


  0%|          | 0/1000 [00:00<?, ?it/s]

k + 1n:	51
k + 2n:	949

k = 1 mod 3


  0%|          | 0/1000 [00:00<?, ?it/s]

k + 1n:	39
k + 2n:	961

k = 2 mod 3


  0%|          | 0/1000 [00:00<?, ?it/s]

k + 1n:	37
k + 2n:	963
